# Sentiment Analysis using Article Headlines from FinViz.com

In [1]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup

In [2]:
# Create a URL string and a list of tickers
finviz_url = "https://www.finviz.com/quote.ashx?t="
tickers = ["ROKU"]

news_tables = {} 

# Loop through the tickers to combine with the URL to make one string representing a completed URL
for ticker in tickers:
    url = finviz_url + ticker
    
    # Request the data from URL (specify the user agent for headers)
    request = Request(url=url, headers={'user-agent': 'my-app'})
    response = urlopen(request)
    
    # Input the response data into BeautifulSoup specifying parse as html
    html = BeautifulSoup(response, 'html')
    
    # Get the HTML objects of the table that contain the article headlines and assign to each ticker in the dictionary
    news_table = html.find(id = 'news-table')
    news_tables[ticker] = news_table

In [3]:
parsed_data = []

for ticker, news_table in news_tables.items():
    
    # Getting all the tr elements from each row of the news_tables dictionary
    for row in news_table.findAll('tr'):
        # Get titles in the tag <a ...> 
        title = row.a.text
        # Split the text of date and time based on a space
        date_data = row.td.text.split(' ')
        # If the lenth of date_data is 1 then it is time. If the length is 2, first is date and second is time.
        if len(date_data) == 1:
            time = date_data[0]
        else:
            date = date_data[0]
            time = date_data[1]
        # Append the ticker, date, time, and title that results in a list of list inside the parsed_data
        parsed_data.append([ticker, date, time, title])

# print(parsed_data) 

In [4]:
# Import Pandas & Natural Language Toolkit Library
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [5]:
df = pd.DataFrame(parsed_data, columns=['ticker', 'date', 'time', 'title'])
df.head()

,ticker,date,time,title
0,ROKU,Jan-22-21,02:10PM,Magnite (MGNI) Extends Rally After Price Targe...
1,ROKU,Jan-21-21,05:51PM,Netflix Joins Ranks Of Stocks With 95-Plus Com...
2,ROKU,Jan-20-21,06:44AM,Why Netflix stock is going haywire after its l...
3,ROKU,Jan-19-21,01:55PM,Why Roku Could Still Have 'Significant' Advert...
4,ROKU,Jan-19-21,11:47AM,Roku (ROKU) Makes Highs on Bullish JPMorgan In...


In [6]:
# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

In [7]:
f = lambda title: analyzer.polarity_scores(title)['compound']
df['compound'] = df['title'].apply(f)
df['date'] = pd.to_datetime(df.date).dt.date
df.head()

,ticker,date,time,title,compound
0,ROKU,2021-01-22,02:10PM,Magnite (MGNI) Extends Rally After Price Targe...,0.4939
1,ROKU,2021-01-21,05:51PM,Netflix Joins Ranks Of Stocks With 95-Plus Com...,0.0000
2,ROKU,2021-01-20,06:44AM,Why Netflix stock is going haywire after its l...,0.0000
3,ROKU,2021-01-19,01:55PM,Why Roku Could Still Have 'Significant' Advert...,0.3818
4,ROKU,2021-01-19,11:47AM,Roku (ROKU) Makes Highs on Bullish JPMorgan In...,0.0000


In [8]:
df.to_csv("../Data/roku_sentiment_finviz.csv", index=False)